In [ ]:
using SymPy
using LinearAlgebra

## 1. feladat

### Setup

In [ ]:
# DH matrix
function DH(α, a, d, θ)
    [cos(θ) -sin(θ) 0 a;
    cos(α)*sin(θ) cos(α)*cos(θ) -sin(α) -d*sin(α);
    sin(α)*sin(θ) sin(α)*cos(θ) cos(α) d*cos(α);
    0 0 0 1]
end

In [ ]:
# Mechanism parameters
l10_ = Sym(8//10); #m
l11_ = Sym(6//10); #m
l2_ = Sym(3//10); #m
rhoA_ = Sym(60); #kg/m
m_ = Sym(5); #kg
IE_ = Sym(1); #MNm^2

g_ = Sym(981//100); # m/s^2

In [ ]:
# just for shorter Sym(0)-s
_0 = Sym(0);

In [ ]:
# Symbolic variables

@vars l10 l11 l2
@vars rhoA m IE m10 m11 m2 m20 g m1

In [ ]:
# For derivatives
@vars t
q1 = SymFunction("q1");
q2 = SymFunction("q2");

In [ ]:
function ssub(A, x1, x2)
    x1f = x1.from
    x1t = x1.to
    x2f = x2.from
    x2t = x2.to
    return A.subs(x1f, x1t).subs(x2f, x2t)
end

In [ ]:
#x1t = (from = q1, to = q1t(t))
#x2t = (from = q2, to = q2t(t))
#t1x = (from = q1t(t), to = q1)
#t2x = (from = q2t(t), to = q2)
dt1 = (from = q1'(t), to = Sym(1))
dt2 = (from = q2'(t), to = Sym(1))

In [ ]:
function d(A, x)
    A_ = deepcopy(A)
    for i in eachindex(A)
        A_[i] = diff(A[i], x)
    end
    A_
end

In [ ]:
function simpl(A)
    A_ = deepcopy(A)
    for i in eachindex(A)
        A_[i] = simplify(A[i])
    end
    A_
end

In [ ]:
J(V, x1, x2) = hcat(Sym[diff(V[i], x1) for i in 1:3], Sym[diff(V[i], x2) for i in 1:3])

In [ ]:
function steiner(m, x,y,z)
    m*[y^2+z^2 -x*y -x*z;
    -y*x x^2+z^2 -y*z;
    -z*x -z*y x^2+y^2]
end

In [ ]:
function steiner(m,v)
    steiner(m, v[1], v[2], v[3])
end

In [ ]:
function mB(v, dx)
    r = [diff(v[i], dx[1]) for i in 1:length(v)]
    for j in 2:length(dx)
        r = hcat(r, [diff(v[i], dx[j]) for i in 1:length(v)])
    end
    r
end

### DH paraméterek

In [ ]:
# first joint
α1 = _0; a1 = _0; d1 = l10; θ1 = PI/2+q1(t);
# second joint
α2 = -PI/2; a2 = l11; d2 = _0; θ2 = q2(t);
# E TCP
α3 = PI/2; a3 = _0; d3 = l2; θ3 = -PI/2;

In [ ]:
_0T1 = DH(α1, a1, d1, θ1)

In [ ]:
_1T2 = DH(α2, a2, d2, θ2)

In [ ]:
_2TTCP = DH(α3, a3, d3, θ3)

### Direkt kinematika

In [ ]:
_2rTCP = [0, -l2, 0, 1]

In [ ]:
_0rTCP = _0T1 * _1T2 * _2rTCP

In [ ]:
_0rTCP_check = _0T1 * _1T2 * _2TTCP * [0,0,0,1]

In [ ]:
_0rTCP_check == _0rTCP

In [ ]:
# Hasznos mátrixok
_0T2 = _0T1 * _1T2;

### Inverz kinematika

`_0rTCP` - ből

### J_E Jacobi-mátrix

In [ ]:
J_E = J(_0rTCP, q1(t), q2(t))

### Forgatási tengely és szög

In [ ]:
_0TTCP = _0T1 * _1T2 * _2TTCP

In [ ]:
_0RTCP_q20 = _0TTCP.subs(q2(t), Sym(0))[1:3, 1:3]

## 2. feladat

## Tömegek

In [ ]:
m10_ = rhoA_*l10_;
m11_ = rhoA_*l11_;
m1_ = rhoA_*(l10_+l11_);
m20_ = rhoA_*l2_;
m2_ = m20_+m_;

#### Sebesség Jacobi

In [ ]:
# rúd10
_1r01 = [_0, _0, -l10/2, Sym(1)];
# rúd11
_1r11 = [l11/2, _0, _0, Sym(1)];
# rúd2
_2r2 = [_0, -l2/2, _0, Sym(1)];
# m tömeg
_2rm = [_0, -l2, _0, Sym(1)];

In [ ]:
# transzformálás 0-s kr-be és összegzés a tagokra
_1rrud1 = (m10_*_1r01+m11_*_1r11)/m1_;
_0rrud1 = _0T1*_1rrud1;
_2rrud2 = (m20_*_2r2+m_*_2rm)/m2_;
_0rrud2 = _0T2*_2rrud2;

In [ ]:
_0Jrs1 = J(_0rrud1, q1(t), q2(t));
N0Jrs1 = _0Jrs1.subs(l11, l11_);
N0Jrs1.evalf(4)

In [ ]:
_0Jrs2 = J(_0rrud2, q1(t), q2(t));
N0Jrs2 = _0Jrs2.subs(l11, l11_).subs(l2, l2_);
N0Jrs2.evalf(5)

#### Szögsebesség Jacobi

##### ω1

In [ ]:
_0R1 = _0T1[1:3,1:3]

In [ ]:
_0R1d = d(_0R1,t)

In [ ]:
_0ωx1 = _0R1d * permutedims(_0R1);
_ω1 = simpl([_0ωx1[3,2], _0ωx1[1,3], _0ωx1[2,1]])

In [ ]:
_0Jω1 = simpl(J(_ω1, q1'(t), q2'(t)))

##### ω2

In [ ]:
_0R2 = _0T2[1:3,1:3]

In [ ]:
_0R2d = d(_0R2, t);
simpl(_0R2d)

In [ ]:
_0ωx2 = _0R2d * permutedims(_0R2);
_ω2 = simpl([_0ωx2[3,2], _0ωx2[1,3], _0ωx2[2,1]])

In [ ]:
_0Jω2 = simpl(J(_ω2, q1'(t), q2'(t)))

### Tehetetlenségi nyomatékok

#### 1-es test

In [ ]:
_1th10 = m10*[Sym(1//12)*l10^2 _0 _0;
            _0 Sym(1//12)*l10^2 _0;
            _0 _0 _0]

In [ ]:
_1th10s = steiner(m10, _1r01-_1rrud1)

In [ ]:
_1th11 = m11*[_0 _0 _0;
            _0 Sym(1//12)*l11^2 _0;
            _0 _0 Sym(1//12)*l11^2]

In [ ]:
_1th11s = steiner(m11, _1r11-_1rrud1)

In [ ]:
_1θ1s = _1th10+_1th10s+_1th11+_1th11s
simpl(_1θ1s)

In [ ]:
N_1θ1s = _1θ1s.subs(m10,m10_).subs(m11,m11_).subs(l10,l10_).subs(l11,l11_);
N_1θ1s.evalf(5)

#### 2-es test

In [ ]:
_2th20 = m20*[Sym(1//12)*l2^2 _0 _0;
            _0 _0 _0;
            _0 _0 Sym(1//12)*l2^2]

In [ ]:
_2th20s = steiner(m20, _2r2-_2rrud2)

In [ ]:
_2thms = steiner(m, _2rm-_2rrud2)

In [ ]:
_2θ2s = _2th20+_2th20s+_2thms;
simpl(_2θ2s)

In [ ]:
N_2θ2s = _2θ2s.subs(m20,m20_).subs(m,m_).subs(l2,l2_)
N_2θ2s.evalf(5)

### Tömegmátrixok

In [ ]:
Hv1 = m1_*permutedims(N0Jrs1)*N0Jrs1;
simpl(Hv1)

In [ ]:
Hv2 = m2_*permutedims(N0Jrs2)*N0Jrs2;
simpl(Hv2)

In [ ]:
Hω1 = permutedims(_0Jω1) * _0R1 * _1θ1s * permutedims(_0R1) * _0Jω1

In [ ]:
Hω2 = permutedims(_0Jω2) * _0R2 * _2θ2s * permutedims(_0R2) * _0Jω2;
simpl(Hω2)

In [ ]:
H = Hv1+Hv2+Hω1+Hω2;
simpl(H)

In [ ]:
simpl(H.subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_)).evalf(5)

In [ ]:
H = Hv1+Hv2+Hω1+Hω2;
simpl(H)

In [ ]:
simpl(H.subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_).evalf(5))

### Kinetikus energia

In [ ]:
T_ = Sym(1//2)*[q1'(t) q2'(t)]*H*[q1'(t), q2'(t)];
T = T_[1].subs(m10,m10_).subs(m20,m20_).subs(m11,m11_).subs(m,m_).subs(l11,l11_).subs(l2,l2_);

In [ ]:
simplify(T.evalf(5))

### Potenciális energia

In [ ]:
U_ = m1_*[0 0 g]*_0rrud1[1:3] + m2_*[0 0 g]*_0rrud2[1:3];
U = U_[1].subs(l10,  l10_).subs(g, g_).subs(l2, l2_);
simplify(U).evalf(5)

### Másodfajú Lagrange-egyenlet

##### k=1

In [ ]:
deltaT_deltaq1p = simplify(diff(T, q1'(t)).evalf(5))

In [ ]:
d_dt_deltaT_deltaq1p = simplify(diff(deltaT_deltaq1p, t)).subs(cos(q2(t)), _0)

##### k=2

In [ ]:
deltaT_deltaq2p = simplify(diff(T, q2'(t), t)).evalf(5)

In [ ]:
deltaT_deltaq2 = simplify(diff(T, q2(t)).evalf(5))

In [ ]:
deltaU_deltaq2 = diff(U, q2(t)).evalf(5)

## 3. feladat

### Kinematikai kényszerek

In [ ]:
# 1-es test súlyponti sebessége
vs1_ = _ω1 × _0rrud1[1:3];
vs1 = vs1_.subs(l11, l11_);
vs1.evalf(5)

In [ ]:
# 2-es test súlyponti sebessége
vs2_ = _ω2 × _0rrud2[1:3];
vs2 = simpl(vs2_.subs(l11, l11_).subs(l10, l10_).subs(l2, l2_));
vs2.evalf(5)

In [ ]:
vs1x = SymFunction("vs1x");
vs1y = SymFunction("vs1y");
vs1z = SymFunction("vs1z");
ω1x = SymFunction("ω1x");
ω1y = SymFunction("ω1y");
ω1z = SymFunction("ω1z");
vs2x = SymFunction("vs2x");
vs2y = SymFunction("vs2y");
vs2z = SymFunction("vs2z");
ω2x = SymFunction("ω2x");
ω2y = SymFunction("ω2y");
ω2z = SymFunction("ω2z");

In [ ]:
y = [vs1x(t), vs1y(t), ω1z(t), vs2x(t), vs2y(t), vs2z(t), ω2x(t), ω2y(t), ω2z(t)];

In [ ]:
f_ = [ vs1x(t) - vs1[1],
        vs1y(t) - vs1[2],
        vs2x(t) - vs2[1],
        vs2y(t) - vs2[2],
        vs2z(t) - vs2[3],
        ω1z(t) - ω2z(t),
        ω2x(t)*sin(q1(t)) - ω2y(t)*cos(q1(t))];
f = f_.subs(q1'(t), ω1z(t)).subs(q2'(t), ω2z(t));
f.evalf(5)

In [ ]:
B = simpl(mB(f, y))

### Gibbs

```julia
_0θ1s = simpl((_0R1 * _1θ1s * permutedims(_0R1)).subs(l10, l10_).subs(l11, l11_).subs(m10, m10_).subs(m11, m11_))

_2θ2s

_0θ2s_ = _0R2 * _2θ2s * permutedims(_0R2);
_0θ2s = _0θ2s_.subs(l2, l2_).subs(m, m_).subs(m20, m20_)

_0θ2s.evalf(5)



_Vs1 = [vs1x(t), vs1y(t), vs1z(t)];
_Vs2 = [vs2x(t), vs2y(t), vs2z(t)];
#_ωs1 = [ω1x(t), ω1y(t), ω1z(t)];
#_ωs2 = [ω2x(t), ω2y(t), ω2z(t)];
_ωs11 = [ω1x(t), ω1y(t), q1'(t)];
_ωs22 = [ω2x(t), ω2y(t), q2'(t)];
y2_ = vcat(_Vs1, _ωs11, _Vs2, _ωs22);

y2p = [diff(yp, t) for yp in y2_];

gt = [1,2, collect(6:12)...];

y2s = y2_[gt]

y2ps = y2p[gt]



Gv1s = (Sym(1//2) * m1 * permutedims(d(_Vs1, t)) * d(_Vs1, t))[1]

Gv2s = simpl(Sym(1//2) * (m20+m) * permutedims(d(_Vs2, t)) * d(_Vs2, t))[1]

Gω1s = (Sym(1//2)* permutedims(d(_ω1, t)) * _0θ1s * d(_ω1, t))[1]

Gω2s_ = Sym(1//2)* permutedims(d(_ωs22, t)) * _0θ2s * d(_ωs22, t)

Gω2s = Gω2s_[1];

Gom2s = simplify(Gω2s)

simpl([diff(Gom2s, yp) for yp in y2ps])



Gωω2_ = permutedims(d(_ωs22, t)) * ( _ωs22 × ( _0θ2s * _ωs22))
Gωω2 = simplify(Gωω2_[1])

simpl([diff(Gωω2, yp) for yp in y2ps])





Gv1s.as_coefficients_dict()

Gv1s

HMH = simpl([diff(Gv1s+Gv2s+Gωω2+Gω2s+Gω1s, diff(yp, t)) for yp in y2ps]);

HMH[end].as_coefficients_dict()



[HMH[end].coeff(yp) for yp in y2s]

HMH[end].coeff(y2[end-2])

_0θ2s





G_ = Gv1s+Gv2s+Gωω2+Gω2s+Gω1s;
simplG = simplify(G_);
trigG = sympy.trigsimp(G_);

simplG

y2ps

G_.coeff(y2ps[7]).coeff(y2ps[7])

G_.coeff(q2''(t)).coeff(ω2y(t))



ms = [m1_, m1_, _0θ1s[3,3], m2_, m2_, m2_]

M77 = Diagonal(ms)

Mke = hcat(vcat(M77, fill(_0, 3,6)),vcat(fill(_0, 6,3), _0θ2s))

Mke * y2ps

Gm_ = (Gv1s+Gv2s+Gωω2+Gω2s+Gω1s).subs(m1, m1_).subs(m, m_).subs(m20, m20_);

Gm = simplify(Gm_)

Hm = [diff(Gm, yp) for yp in y2ps];

probablyC = (Hm - Mke * y2ps)

simplC = simplify.(probablyC)

simplC[9]

simplC[7].coeff(q2'(t))

#sympy.expand_trig


@vars C1 S1 C2 S2

# Prohi θs2-j
_6 = Sym(6745/10000)
theta_prohi = [_6*C1^2+_6*C2^2*S1^2 (_6-_6*C2^2)*S1*C1 _6*S1*S2*C2;
                (_6-_6*C2^2)*S1*C1 _6*C1^2*C2^2+_6*S1^2 -_6*C1*C2*S2
                _6*S1*S2*C2 -_6*C1*C2*S2 _6*S2^2]

simpl(theta_prohi)
```